## Machine Learning intro
- Supervised learning
  - Models designed to infer a relationship between input and **labeled** data.      
  - We define the **target** as labels in data that we wish to model (e.g.  population as a function of year)</p>
- Unsupervised learning
  - Find patterns and relationships from **unlabeled** data 
  - This may involve clustering (e.g. group objects that share certain degree of similarity), dimensionality reduction and more.  
  - *Not part of the course*.</p>
- Reinforcement learning
  - Models to infer optimal behavior in some (potentially) mathematically unknown environment
  - Needs no labeling, and suboptimal behavior is corrected through experience (penalty/reward).
  - *Not part of the course*.

### Supervised learning
- Regression:
    - Predict a numeric value (the target can be thought of as ’continuous’)
    - Target data type will typically be float
    - Example of targets:  income, life expectancy, education length (years)


- Classification:
    - Distinguish between non-numeric target categories (the target is binary or a catagory/factor)
    - Target data type can be be str, bool or int
    - Examples of target:  kind of education (linguistics, math), mode of transportation

In [59]:
import numpy as np
import pandas as pd
import seaborn as sns

titanic = sns.load_dataset('titanic')
cols = ['survived','class', 'sex', 'sibsp', 'age', 'alone']
titanic_sub = pd.get_dummies(titanic[cols].dropna(), drop_first=True).astype(np.int64)  

X = titanic_sub.drop('survived', axis=1)
y = titanic_sub.survived

In [60]:
X = np.random.normal(size=(3, 2)) # feature matrix
y = np.array([1, -1, 1]) # target vector
w = np.random.normal(0,0.001, size=(3)) # weight vector
print('X:\n',X)
print('y:',y)
print('w:',w)


X:
 [[-1.18353385  0.19659839]
 [-0.00357423  0.74207524]
 [ 2.07870704 -1.29935775]]
y: [ 1 -1  1]
w: [-0.00022192  0.00026297  0.00086224]


In [82]:
# compute net-input 
z = w[0] + X.dot(w[1:]) # (w[0]: bias, w[1:]: other weights, X: features)  w第一项+X矩阵点乘w的第一项和后面所有项

# unit step-function
positive = z>0 # compute prediction (boolean)
y_hat = np.where(positive, 1, -1)  # convert prediction  if z>0, 赋1，if z<=0, 赋-1

# compute errors
e = y - y_hat # compute errors

In [84]:
# learning rate
eta = 0.001 

# update weights 
w[1:] += eta*X.T.dot(e) 
w[0] += eta*e.sum()


In [85]:
X = np.random.normal(size=(3, 2)) # feature matrix
y = np.array([1, -1, 1]) # target vector
w = np.random.normal(0,0.001, size=(3)) # weight vector

# learning rate
eta = 0.0001 

error = np.inf  # 无限位小数
k = 1

while error>0:
    z = w[0] + X.dot(w[1:])
    positive = z>0
    
    y_hat = np.where(positive, 1, -1)
    e = y - y_hat
    
    w[1:] += eta*X.T.dot(e) 
    w[0] += eta*e.sum()
    
    error = abs(y-y_hat).sum()    
    
    if k == 1:
        print(k,y-y_hat, w)
        
    k += 1

print(k-1,y-y_hat, w)

1 [2 0 2] [-0.00015755  0.00031254  0.00011137]
4 [0 0 0] [ 4.24549242e-05 -2.19072479e-04  2.57036208e-04]


Help on float object:

class float(object)
 |  float(x=0, /)
 |  
 |  Convert a string or number to a floating point number, if possible.
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __ceil__(self, /)
 |      Return the ceiling as an Integral.
 |  
 |  __divmod__(self, value, /)
 |      Return divmod(self, value).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __float__(self, /)
 |      float(self)
 |  
 |  __floor__(self, /)
 |      Return the floor as an Integral.
 |  
 |  __floordiv__(self, value, /)
 |      Return self//value.
 |  
 |  __format__(self, format_spec, /)
 |      Formats the float according to format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getnewargs__(self, /)
 |  
 |  __gt__(self, value, /)
 